In [ ]:
import os
import time
import urllib.request as req
import undetected_chromedriver as uc
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

if __name__ == "__main__":
    driver = uc.Chrome(use_subprocess=False)
    driver.get('https://flight.settour.com.tw/index.html')
    driver.maximize_window()
    time.sleep(2)
    # 往下滑
    driver.execute_script("window.scrollTo(0, 300);")
    # 選到出發的框框點擊
    e = driver.find_element(By.ID, "pfpRTDepAirportInput")
    e.click()
    time.sleep(1)
    # 他跳出的那個下拉的每個選項
    e_list = driver.find_elements(By.CLASS_NAME, "searchbar-select-list-li-des")
    # 選擇松山點擊
    e_list[1].click()

    # 找到回來的那個框框
    e = driver.find_element(By.ID, "pfpRTArrAirportInput")
    e.click()
    time.sleep(1)
    # 如果你直接選, 會選到出發的那個下拉選單
    # 先找到包後面64的那個大區塊
    e_list = (driver.find_element(By.ID, "pfpRTArrAirportInputB2_list")
              .find_elements(By.CLASS_NAME, "searchbar-select-list-li-des"))
    # 因為我們選的出發是台灣, 上面的5個被隱藏了(01234) 5東京開始
    e_list[5].click()

    # 搜尋按鈕的點擊
    e = driver.find_element(By.ID, "pfpRTSearchFormBtn")
    e.click()

    time.sleep(10)

    # 找到每一個商品
    # 幫我拿 航司 出發time 抵達time 價格
    # 整理成一個 CSV
    table = []
    products = driver.find_elements(By.CLASS_NAME, "product-item")
    for product in products:
        # 拿到每一個 準備成字典 放進table

        # 航司
        airline = product.find_element(By.CLASS_NAME, "product-item-flight-name-tit")

        # 出發時間和抵達時間
        time_elements = product.find_elements(By.CLASS_NAME, "station-info-time")
        departure_time = time_elements[0]
        arrival_time = time_elements[1]

        # 價格
        price = product.find_element(By.CLASS_NAME, "price-uni")

        # 組成字典
        flight_data = {
            "航空公司": airline.text,
            "出發時間": departure_time.text,
            "抵達時間": arrival_time.text,
            "價格": price.text
        }
        table.append(flight_data)

        print(f"航司: {airline.text}, 出發: {departure_time.text}, 抵達: {arrival_time.text}, 價格: {price.text}")

    # 儲存成CSV
    df = pd.DataFrame(table)
    df.to_csv('flight_data.csv', index=False, encoding='utf-8')
    print(f"已儲存 {len(table)} 筆資料")

    time.sleep(3)